<a href="https://colab.research.google.com/github/RKC3000/Object-Detection/blob/main/3_Day_Object_Detection_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Object Detection Challenge**

---

**YOLOv5** setup

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-140-g1db9533 Python-3.9.16 torch-2.0.0+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.4/107.7 GB disk)


**Download Dataset**

In [2]:
!bash data/scripts/get_coco.sh

######################################################################## 100.0%
######################################################################## 100.0%
######################################################################## 100.0%


In [12]:
from torchvision.utils import Image
import os
import pandas as pd
from torchvision.io import read_image

class obj_dec(Dataset):
  def __init__(self,image_folder,annotations_file,transform = None):
    self.image_folder = image_folder
    self.annotations = pd.read_csv(annotations_file)
    self.transform = transform
  
  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, idx):

    # Load the image and convert to RGB
    image_path = os.path.join(self.image_folder, self.annotations.iloc[idx,0])
    image = Image.open(image_path).convert('RGB')

    # Get the annotations for this image
    annotations = self.annotations.iloc[idx,1:]

    # Convert annotations to tensor
    annotations = torch.tensor(annotations.astype('float32'))

    # Apply transform if available
    if self.transform:
      image = self.transform(image)

      return image, annotations

  transform = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
  ])

NameError: ignored

In [ ]:
imagenet_data = torchvision.datasets.ImageNet('/content/datasets/coco128/images/')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=args.nThreads)